In [ ]:
#Colab users uncomment this code
#Files for this notebook are Energy Indicators.xls and world_bank.csv

In [ ]:
"""
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
"""

In [ ]:
"""
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
"""

In [ ]:
"""
downloaded = drive.CreateFile({'id':'1R9vW5dmox7i8OGOoDql_r9yUrvAJgvhD'}) # replace the id with id of file you want to access
downloaded.GetContentFile('Energy Indicators.xls')
"""

In [ ]:
"""
downloaded = drive.CreateFile({'id':'1R9vW5dmox7i8OGOoDql_r9yUrvAJgvhD'}) # replace the id with id of file you want to access
downloaded.GetContentFile('world_bank.csv')
"""

# PANDAS ASSIGNMENT SOLUTIONS (100)
<hr style="height:5px;border-width:2;color:gray">

In this assignment you'll be handling two datasets, <code>Energy Indicators.xls</code> and <code>world_bank.csv</code> and answering some questions.<br><br>
<b>Grading:</b>
<ul>
    <li>Each question has the corresponding total marks next to it.</li>
    <li>Each question will be graded on <b>final answer (60%)</b> and <b>conciseness of code (40%)</b>.</li>
    <li>There is no partial marking for final answer but there is partial marking for conciseness of code</li>
    <li>The total marks will be scaled down later on as per requirement.</li>
</ul>

In [1]:
import pandas as pd
import numpy as np

## Loading and Cleaning the Data (50)
<div class="alert alert-block alert-warning">
    <b>Note that this is the most important part of this assignment as without successfully loading and cleaning the data, future questions cannot be answered.</b>
</div>

#### Energy Indicators data ( 6 X 5 = 30)
<b>Instructions</b>:
<ol>
    <li>Load the <b>excel file</b> <code>Energy Indicators.xls</code> into dataframe called <code>energy</code>.</li>
    <li>Make sure to exclude header and footer information from the datafile.</li>
    <li>The first two columns are unecessary and hence get rid of them.</li>
    <li>Change the remaining column names to <code>['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']</code>.</li>
    <li>The Energy Supply is in petajoule. Covert it to gigajoule. (1 petajoule = 1,000,000 gigajoule)</li>
    <li>Rename the following list of countries:
        <ul>
        <li>"Republic of Korea" : "South Korea"</li>
        <li>"United States of America" : "United States"</li>
        <li>"United Kingdom of Great Britain and Northern Ireland" : "United Kingdom"</li>
        <li>"China, Hong Kong Special Administrative Region" : "Hong Kong"</li>
        </ul>
    </li>
</ol>
<div class="alert alert-block alert-info">
    <b>Hint</b>: <code>energy</code> dataframe has 227 rows and 4 columns.
</div>

In [2]:
energy = pd.read_excel("Energy Indicators.xls", skiprows = 17, skipfooter = 38, usecols = range(2,6), names = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable'])
energy["Energy Supply"] = energy["Energy Supply"]*1000000
di = {"Republic of Korea": "South Korea", "United States of America": "United States", "United Kingdom of Great Britain and Northern Ireland": "United Kingdom", "China, Hong Kong Special Administrative Region": "Hong Kong"}
energy['Country'].replace(di, inplace = True)
energy

FileNotFoundError: [Errno 2] No such file or directory: 'Energy Indicators.xls'

#### GDP Data (10)
<b>Instructions: </b><br>
<ol>
    <li>Load the <b>csv file</b> <code>world_bank.csv</code> into a dataframe called <code>GDP</code>.</li>
    <li>Skip the headers.</li>
    <li>The GDP dataframe must only have the columns of country name and GDP of years 2006-2015 (both inclusive)</li>
    <li>Rename the following list of countries:</li>
    <ul>
        <li>"Korea, Rep.": "South Korea"</li>
        <li>"Iran, Islamic Rep.": "Iran"</li>
        <li>"Hong Kong SAR, China": "Hong Kong"</li>
    </ul>
</ol>
<div class="alert alert-block alert-info">
    <b>Hint</b>: <code>GDP</code> dataframe has 264 rows and 11 columns.
</div>

In [3]:
GDP = pd.read_csv("world_bank.csv", skiprows = 4)
di = {"Korea, Rep.": "South Korea", "Iran, Islamic Rep.": "Iran", "Hong Kong SAR, China": "Hong Kong"}
GDP['Country Name'].replace(di, inplace = True)
cols = [GDP.columns[0]] + list(GDP.columns[-10:]) #Or usecols attribute can be used in pd.read_csv
GDP = GDP[cols]
GDP

FileNotFoundError: [Errno 2] File world_bank.csv does not exist: 'world_bank.csv'

#### Merging the dataframes (10)
<b>Instructions</b>:
Merge the two dataframes into one and call it <code>energy_GDP</code> which has data pertaining to countries common to both dataframes. <code>energy_GDP</code> must have <b>13 columns</b> excluding the index, which should be <code>Country</code>. After this reduce the dataframe to have data pertaining the countries given in the next cell.

In [ ]:
countries = ['China', 'United States', 'Japan', 'United Kingdom', 'Russian Federation', 'Canada', 'Germany', 'India', 'France', 'South Korea', 'Italy', 'Spain', 'Iran', 'Australia', 'Brazil']

In [ ]:
energy_GDP = pd.merge(energy, GDP.rename(columns = {'Country Name':'Country'}), how = 'inner', on = 'Country').set_index('Country')
energy_GDP = energy_GDP.loc[countries]
energy_GDP

## Questions (50)

#### 1. What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.) (5)
(Your answer should be a series in descending order by average GDP)

In [ ]:
avgGDP = energy_GDP.iloc[:, -10:].mean(axis = 1)
avgGDP = avgGDP.sort_values(ascending = False)
avgGDP

#### 2. By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP? (5)
(Your answer should be a single number)

In [ ]:
ctry = avgGDP.index[5]
energy_GDP.loc[ctry, '2015'] - energy_GDP.loc[ctry, '2006']

#### 3. What country has the maximum % Renewable and what is the percentage? (10)
(Your answer should be a python tuple coontaining country name and its % Renewable)

In [ ]:
(energy_GDP["% Renewable"].idxmax(), energy_GDP["% Renewable"].max())

#### 4(i). Create a column that estimates the population using Energy Supply and Energy Supply per capita. (5)
<div class="alert alert-block alert-warning">
    <b>Note: </b>Convert the population estimation by energy to np.float64 format.
</div>
(Your answer should be the first five rows of a dataframe)


In [ ]:
energy_GDP["Population Estimate by Energy"] = np.float64(energy_GDP["Energy Supply"]/energy_GDP["Energy Supply per Capita"])
energy_GDP.head()

#### 4(ii). What is the third most populous country according to this (5)
(Your answer should be a single string)

In [ ]:
energy_GDP.sort_values(by = 'Population Estimate by Energy', ascending = False).index[2]

#### 5. Create a series with country as index and value 1 if the country's % Renewable is above the median % Renewable of the 15 countries and 0 otherwise. (5)
(Your answer should be a series)

In [ ]:
med = energy_GDP["% Renewable"].median()
HighRenew = (energy_GDP["% Renewable"] >= med)*1
HighRenew

#### 6. Summary of continents (15)
Group the countries by continent and display the following data for each continent: (All data regarding the given 15 countries)
<ul>
    <li><b>size</b>: The number of countries in this continent</li>
    <li><b>sum</b>: Total population estimate of this continent</li>
    <li><b>mean</b>: Mean country population in this continent</li>
    <li><b>std</b>: Standard deviation of country population in this continent</li>
</ul>
Use the dictionary in the next cell to assign corresponding continent to each country.<br>
(Your answer should be  a dataframe with index named Continent <code>['Asia', 'Australia', 'Europe', 'North America', 'South America']</code> and columns <code>['size', 'sum', 'mean', 'std']</code> and it will have two <code>NaN</code>)

In [ ]:
continents = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}

In [ ]:
Continents_df=energy_GDP['Population Estimate by Energy'].groupby(continents).agg(['size','sum','mean','std'])
Continents_df

### Sources:
<ul>
    <li><b>Exercise</b>: Some questions in the exercise were taken from Udemy's Python for Data Science and Machine Learning Bootcamp course.</li>
    <li><b>Assignment</b>: The dataset and assignment questions were taken from Coursera's Inroduction to Data Science in Python course (University of Michigan). However the dataset was slightly modified to make the cleaning process easier.</li>